## User Price Predction

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
df= pd.read_csv('cardekho_dataset.csv')
df.head()

,Unnamed: 0,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


## Data Cleaning

In [3]:
## checking null values
df.isnull().sum()

Unnamed: 0           0
car_name             0
brand                0
model                0
vehicle_age          0
km_driven            0
seller_type          0
fuel_type            0
transmission_type    0
mileage              0
engine               0
max_power            0
seats                0
selling_price        0
dtype: int64

In [4]:
# remove unwanter columns
df.drop('car_name', axis=1, inplace=True)
df.drop('brand', axis=1, inplace= True)

In [5]:
df.head()

,Unnamed: 0,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [6]:
df['model'].unique()

array(['Alto', 'Grand', 'i20', 'Ecosport', 'Wagon R', 'i10', 'Venue',
       'Swift', 'Verna', 'Duster', 'Cooper', 'Ciaz', 'C-Class', 'Innova',
       'Baleno', 'Swift Dzire', 'Vento', 'Creta', 'City', 'Bolero',
       'Fortuner', 'KWID', 'Amaze', 'Santro', 'XUV500', 'KUV100', 'Ignis',
       'RediGO', 'Scorpio', 'Marazzo', 'Aspire', 'Figo', 'Vitara',
       'Tiago', 'Polo', 'Seltos', 'Celerio', 'GO', '5', 'CR-V',
       'Endeavour', 'KUV', 'Jazz', '3', 'A4', 'Tigor', 'Ertiga', 'Safari',
       'Thar', 'Hexa', 'Rover', 'Eeco', 'A6', 'E-Class', 'Q7', 'Z4', '6',
       'XF', 'X5', 'Hector', 'Civic', 'D-Max', 'Cayenne', 'X1', 'Rapid',
       'Freestyle', 'Superb', 'Nexon', 'XUV300', 'Dzire VXI', 'S90',
       'WR-V', 'XL6', 'Triber', 'ES', 'Wrangler', 'Camry', 'Elantra',
       'Yaris', 'GL-Class', '7', 'S-Presso', 'Dzire LXI', 'Aura', 'XC',
       'Ghibli', 'Continental', 'CR', 'Kicks', 'S-Class', 'Tucson',
       'Harrier', 'X3', 'Octavia', 'Compass', 'CLS', 'redi-GO', 'Glanza',
       

In [7]:
## Getting all type of features
num_features = [feature for feature in df.columns if df[feature].dtype !='O'] 
print('Num of Numerical Features : ', len(num_features))

cat_features = [feature for feature in df.columns if df[feature].dtype =='O'] 
print('Num of Categorical Features : ', len(cat_features))

discrete_features = [ feature for feature in num_features if len(df[feature].unique())<=25]
print('Num of discrete Features : ', len(discrete_features))

continuous_features = [ feature for feature in num_features if feature not in discrete_features]
print('Num of discrete Features : ', len(continuous_features))



Num of Numerical Features :  8
Num of Categorical Features :  4
Num of discrete Features :  2
Num of discrete Features :  6


In [8]:
from sklearn.model_selection import train_test_split
X= df.drop(['selling_price'], axis=1)
y=df['selling_price']

## Feature Encoding and Scaling

In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X['model'] = le.fit_transform(X['model'])

In [10]:
X.dtypes

Unnamed: 0             int64
model                  int64
vehicle_age            int64
km_driven              int64
seller_type           object
fuel_type             object
transmission_type     object
mileage              float64
engine                 int64
max_power            float64
seats                  int64
dtype: object

In [11]:
X['model'].unique()

array([  7,  54, 118,  38, 100, 117,  96,  88,  97,  32,  29,  24,  16,
        60,  13,  89,  95,  30,  25,  14,  45,  64,  10,  84, 114,  63,
        59,  78,  85,  68,  11,  44,  98,  91,  73,  86,  23,  49,   1,
        19,  41,  62,  61,   0,   4,  92,  42,  83,  90,  58,  79,  39,
         5,  36,  74, 116,   2, 111, 106,  57,  26,  31,  22, 103,  77,
        46,  87,  70, 113,  34,  82,  99, 112,  93,  37, 101,  20,  40,
       115,  47,   3,  81,  33,  12, 107,  51,  28,  18,  65,  80,  94,
        56, 104,  71,  27,  17, 119,  53,  67, 105,  35, 109,  43,   6,
        66,  50,  48, 102, 110, 108,  72,   9,   8,  69,  21,  15,  76,
        52,  75,  55])

In [12]:
# create column Transformer with 3 types of transformer
num_features = X.select_dtypes(exclude='object').columns
print(num_features)
onehot_columns = ['seller_type', 'fuel_type', 'transmission_type']
label_encoder_columns = ['model']

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder(drop='first')

preprocessor = ColumnTransformer([
    ('OneHotEncoder', oh_transformer, onehot_columns),
    ('StandardScaler',numeric_transformer, num_features)
],remainder='passthrough')

Index(['Unnamed: 0', 'model', 'vehicle_age', 'km_driven', 'mileage', 'engine',
       'max_power', 'seats'],
      dtype='object')


In [13]:
X.head()

,Unnamed: 0,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
0,0,7,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5
1,1,54,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5
2,2,118,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5
3,3,7,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5
4,4,38,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5


In [14]:
X= preprocessor.fit_transform(X)

In [15]:
pd.DataFrame(X)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,-1.738694,-1.519714,0.983562,1.247335,-0.000276,-1.324259,-1.263352,-0.403022
1,1.0,0.0,0.0,0.0,0.0,1.0,1.0,-1.738516,-0.225693,-0.343933,-0.690016,-0.192071,-0.554718,-0.432571,-0.403022
2,1.0,0.0,0.0,0.0,0.0,1.0,1.0,-1.738339,1.536377,1.647309,0.084924,-0.647583,-0.554718,-0.479113,-0.403022
3,1.0,0.0,0.0,0.0,0.0,1.0,1.0,-1.738162,-1.519714,0.983562,-0.360667,0.292211,-0.936610,-0.779312,-0.403022
4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,-1.737985,-0.666211,-0.012060,-0.496281,0.735736,0.022918,-0.046502,-0.403022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15406,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.723327,1.508844,0.983562,-0.869744,0.026096,-0.767733,-0.757204,-0.403022
15407,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.723859,-0.556082,-1.339555,-0.728763,-0.527711,-0.216964,-0.220803,2.073444
15408,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.724036,0.407551,-0.012060,0.220539,0.344954,0.022918,0.068225,-0.403022
15409,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.724213,1.426247,-0.343933,72.541850,-0.887326,1.329794,0.917158,2.073444


In [16]:
## train test split
X_train, X_test, y_train, y_test= train_test_split(X,y,test_size=0.2, random_state=10)

## Model Training and Model Selection

In [17]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [18]:
## create a function to evaluate Model
def evaluate_model(true, predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mean_squared_error(true,predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [20]:
models = {
    'Linear Regression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge': Ridge(),
    'K-Neighbors Regressor': KNeighborsRegressor(),
    'Decision Tree':DecisionTreeRegressor(),
    'Random Forestrest Regressor': RandomForestRegressor()

}

for i in range(len(list(models))):
    print(i)
    model =list(models.values())[i]
    model.fit(X_train,y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # evaluate Train and Test Dataset
    model_train_mae,model_train_rmse,model_train_r2= evaluate_model(y_train,y_train_pred)
    model_test_mae,model_test_rmse,model_test_r2= evaluate_model(y_test,y_test_pred)
    print(list(models.keys())[i])

    print("Model performance for Training set")
    print("-Root Mean Squared Error: {:4f}".format(model_train_rmse))
    print("-Mean Absolute Error: {:4f}".format(model_train_mae))
    print("-R2 Score: {:4f}".format(model_train_r2))

    print('-------------------------------------------------------------------')

    print("Model performance for Testiing set")
    print("-Root Mean Squared Error: {:4f}".format(model_test_rmse))
    print("-Mean Absolute Error: {:4f}".format(model_test_mae))
    print("-R2 Score: {:4f}".format(model_test_r2))

    print('='*35)
    print('\n')

0
Linear Regression
Model performance for Training set
-Root Mean Squared Error: 478801.846874
-Mean Absolute Error: 260031.818718
-R2 Score: 0.676295
-------------------------------------------------------------------
Model performance for Testiing set
-Root Mean Squared Error: 753414.735440
-Mean Absolute Error: 280916.415811
-R2 Score: 0.512257


1
Lasso
Model performance for Training set
-Root Mean Squared Error: 478801.851944
-Mean Absolute Error: 260029.925521
-R2 Score: 0.676295
-------------------------------------------------------------------
Model performance for Testiing set
-Root Mean Squared Error: 753414.004772
-Mean Absolute Error: 280915.288682
-R2 Score: 0.512258


2
Ridge
Model performance for Training set
-Root Mean Squared Error: 478802.543132
-Mean Absolute Error: 260001.344165
-R2 Score: 0.676294
-------------------------------------------------------------------
Model performance for Testiing set
-Root Mean Squared Error: 753427.792683
-Mean Absolute Error: 2808

## Hyperparameter Tuning

In [22]:
# Initialize few parameter for Hypreparameter tuning
knn_params = {'n_neighbors':[2,3,10,20,40,50]}
rf_params = {
    'max_depth':[5,8,15,None,10],
    'max_features':[5,7,'auto',8],
    'min_samples_split':[2,8,15,20],
    'n_estimators':[100,200,500,1000]
}

In [24]:
# Models list for hyperparameter tuning
randomcv_models = [('KNN',KNeighborsRegressor(),knn_params),
                   ('RF',RandomForestRegressor(),rf_params)]

In [32]:
from sklearn.model_selection import RandomizedSearchCV


model_param={}
for name, model, params in randomcv_models:
    random = RandomizedSearchCV(estimator=model,
                                   param_distributions= params,
                                   n_iter =100,
                                   cv=3,
                                   verbose = 2,
                                   n_jobs=-1
                                   )
    random.fit(X_train,y_train)
    model_param[name]=random.best_params_
for model_name in model_param:
    print(f"----------------------------Best Params for {model_name}----------------------------------")
    print(model_param[model_name])

Fitting 3 folds for each of 6 candidates, totalling 18 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits


KeyboardInterrupt: 